In [3]:
import yfinance as yf
import pandas as pd
import numpy as np

class PortfolioManager:
    def __init__(self):
        self.stocks = {}
        self.portfolio = pd.DataFrame()

    def add_stock(self, ticker, weight):
        """ Add a stock to the portfolio with its weight """
        if ticker in self.stocks:
            print(f"{ticker} is already in the portfolio.")
        else:
            stock_data = yf.Ticker(ticker)
            hist = stock_data.history(period="1y")  # Using 1 year of daily data for better correlation analysis
            if not hist.empty:
                self.stocks[ticker] = {'weight': weight, 'data': hist['Close']}
                print(f"{ticker} added to portfolio.")
            else:
                print(f"Failed to retrieve data for {ticker}.")

    def remove_stock(self, ticker):
        """ Remove a stock from the portfolio """
        if ticker in self.stocks:
            del self.stocks[ticker]
            print(f"{ticker} removed from portfolio.")
        else:
            print(f"{ticker} not found in portfolio.")

    def calculate_portfolio(self):
        """ Calculate the weighted returns and risk of the portfolio """
        if not self.stocks:
            print("No stocks in portfolio.")
            return

        stock_data_frames = []
        for ticker, info in self.stocks.items():
            df = pd.DataFrame(info['data'])
            df.columns = [ticker]
            stock_data_frames.append(df)

        combined_data = pd.concat(stock_data_frames, axis=1)
        returns = combined_data.pct_change().dropna()

        # Calculate weighted returns and risks
        weights = np.array([info['weight'] for info in self.stocks.values()])
        mean_returns = returns.mean()
        cov_matrix = returns.cov()

        portfolio_return = np.sum(mean_returns * weights) * 252
        portfolio_std_dev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(252)

        # Calculate and display correlation matrix
        correlation_matrix = returns.corr()
        print("Correlation Matrix:\n", correlation_matrix)
        print(f"Expected Annual Return: {portfolio_return:.2%}")
        print(f"Annual Portfolio Standard Deviation: {portfolio_std_dev:.2%}")

    def invest_amount(self, amount):
        """ Calculate the number of shares to purchase for each stock """
        if self.portfolio.empty:
            print("Calculate the portfolio first.")
            return

        latest_prices = {ticker: data.iloc[-1] for ticker, data in self.stocks.items()}
        total_weight = sum(info['weight'] for info in self.stocks.values())
        allocations = {ticker: (info['weight'] / total_weight) * amount / latest_prices[ticker] for ticker, info in self.stocks.items()}
        
        print("Number of shares to buy for the given investment amount:")
        for ticker, shares in allocations.items():
            print(f"{ticker}: {shares:.2f}")

# Example usage:
portfolio = PortfolioManager()
portfolio.add_stock('AAPL', 0.1)
portfolio.add_stock('TSLA', 0.1)
portfolio.add_stock('VOO', 0.1)
portfolio.add_stock('VGT', 0.1)
portfolio.add_stock('GLD', 0.1)  # Gold ETF
portfolio.add_stock('AMD', 0.1)
portfolio.add_stock('GOOGL', 0.1)
portfolio.add_stock('SLV', 0.3)  # Silver ETF
portfolio.calculate_portfolio()
portfolio.invest_amount(10000)  # Example investment amount in dollars


AAPL added to portfolio.
TSLA added to portfolio.
VOO added to portfolio.
VGT added to portfolio.
GLD added to portfolio.
AMD added to portfolio.
GOOGL added to portfolio.
SLV added to portfolio.
Correlation Matrix:
            AAPL      TSLA       VOO       VGT       GLD       AMD     GOOGL  \
AAPL   1.000000  0.374162  0.621460  0.681051  0.022152  0.317975  0.460867   
TSLA   0.374162  1.000000  0.486152  0.451949 -0.007471  0.257069  0.280013   
VOO    0.621460  0.486152  1.000000  0.889781  0.084250  0.554722  0.551079   
VGT    0.681051  0.451949  0.889781  1.000000  0.014993  0.695933  0.522077   
GLD    0.022152 -0.007471  0.084250  0.014993  1.000000 -0.021062 -0.005183   
AMD    0.317975  0.257069  0.554722  0.695933 -0.021062  1.000000  0.403588   
GOOGL  0.460867  0.280013  0.551079  0.522077 -0.005183  0.403588  1.000000   
SLV    0.099955  0.077257  0.170766  0.107794  0.754487  0.062554  0.036485   

            SLV  
AAPL   0.099955  
TSLA   0.077257  
VOO    0.170766  

In [13]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Retrieve stock data
tickers = ['AAPL', 'TSLA', 'VOO', 'VGT', 'GLD', 'AMD', 'GOOGL', 'SLV']
data = yf.download(tickers, start="2023-01-01", end="2024-04-20")['Close']
returns = data.pct_change().dropna()

# Calculate annual average returns and covariance matrix
annual_returns = returns.mean() * 252
cov_matrix = returns.cov() * 252

# Define objective function (negative Sharpe ratio)
def objective(weights):
    port_return = np.dot(weights, annual_returns)
    port_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe_ratio = port_return / port_volatility
    return -sharpe_ratio

# Constraints: Weights sum to 1 (100% of the portfolio is invested)
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})

# Bounds for weights: Each weight should be between 0 and 1
bounds = tuple((0, 1) for _ in range(len(tickers)))

# Initial guess for the weights
initial_guess = [1. / len(tickers)] * len(tickers)

# Perform optimization
result = minimize(objective, initial_guess, method='SLSQP', bounds=bounds, constraints=constraints)

if result.success:
    optimal_weights = result.x
    print("Optimized Weights:", optimal_weights)
else:
    raise ValueError("Optimization failed:", result.message)

print(tickers)

[*********************100%%**********************]  8 of 8 completed

Optimized Weights: [4.06016097e-16 5.57744050e-02 5.76360265e-01 1.40054363e-01
 1.93331168e-15 0.00000000e+00 2.18559717e-01 9.25124971e-03]
['AAPL', 'TSLA', 'VOO', 'VGT', 'GLD', 'AMD', 'GOOGL', 'SLV']
